In [1]:
!python --version

Python 3.11.11


In [2]:
%pip install transformers
%pip install ipywidgets
%pip install scipy
%pip install torch torchvision torchaudio

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.29.2-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # Should show your CUDA version if installed with CUDA support

2.6.0+cu124
12.4


In [4]:
import torch
# Check if GPU is available
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda:0


In [5]:
model_id = "openai/whisper-small"

In [6]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    chunk_length_s=30,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
)

Device set to use cuda:0


In [28]:
import os
import numpy as np
from scipy.io import wavfile

#filename = "wake_up.wav"
filename = "wake_up_converted.wav"
#filename = "jp-sample.wav"
print(os.path.exists(filename))
# we can't use the "wave" standard library because it can't read 32f files.
# in Adobe Audition you can't export lower than 32f so we'll have to covert.
# We'll use wavefile scipy.io wavfile

# Read the 32-bit float WAV file
sampling_rate, raw = wavfile.read(filename)

True


In [32]:
%%time
task = 'transcribe' # transcribe / translate
transcription = pipe(
    {"raw": raw, "sampling_rate": sampling_rate }, 
    generate_kwargs={"task": task}, 
    return_timestamps="word"
)

/home/andrew/miniconda3/envs/asr-task/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


CPU times: user 1.08 s, sys: 426 ms, total: 1.51 s
Wall time: 1.3 s


In [ ]:
print(transcription)

{'text': 'あなたは新しいアパートにより、朝日の光が窓の中を流すように、あなたは起きた。', 'chunks': [{'text': 'あなたは新しいアパートにより、朝日の光が窓の中を流すように、あなたは起きた。', 'timestamp': (0.0, None)}]}


In [33]:
print(transcription)

{'text': 'あなたは新しいアパートにより、朝日の光が窓の中を流すように、あなたは起きた。', 'chunks': [{'text': 'あなたは新しいアパートにより、朝日の光が窓の中を流すように、あなたは起きた。', 'timestamp': (0.0, None)}]}
